<!-- vscode-jupyter-toc -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->
<a id='toc0_'></a>**Содержание**    
- [Константность](#toc1_)    
  - [Определение констант](#toc1_1_)    
  - [Указатели и const](#toc1_2_)    
    - [Указатель на константу](#toc1_2_1_)    
    - [Константный указатель](#toc1_2_2_)    
    - [Константный указатель на константу](#toc1_2_3_)    
    - [“слово const делает неизменяемым тип слева от него”](#toc1_2_4_)    
  - [Ссылки и const](#toc1_3_)    
  - [Константные методы](#toc1_4_)    
  - [Две версии одного метода](#toc1_5_)    
  - [Cинтаксическая и логическая константность](#toc1_6_)    
  - [Ключевое слово mutable](#toc1_7_)    
- [Конструктор копирования и оператор присваивания](#toc2_)    
  - [Копирование объектов](#toc2_1_)    
    - [Инструменты - статические анализаторы кода](#toc2_1_1_)    
  - [Конструктор копирования](#toc2_2_)    
  - [Оператор присваивания](#toc2_3_)    
  - [Метод swap](#toc2_4_)    
    - [* Идиома копирования и обмена](#toc2_4_1_)    
  - [Реализация оператора = при помощи swap](#toc2_5_)    
  - [Запрет копирования объектов](#toc2_6_)    
  - [Методы, генерируемые компилятором](#toc2_7_)    
- [Правило трёх (также известное как «Закон Большой Тройки» или «Большая Тройка»)](#toc3_)    
  - [Правило пяти](#toc3_1_)    
  - [Правило ноля](#toc3_2_)    
  - [Пример реализации правила пяти](#toc3_3_)    
  - [Неправильные конструкторы присваивания](#toc3_4_)    
- [Класс массива](#toc4_)    
  - [/* Поля и конструкторы */](#toc4_1_)    
  - [/* Деструктор, оператор присваивания и swap */](#toc4_2_)    
  - [/* Остальные методы */](#toc4_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- /vscode-jupyter-toc -->

# <a id='toc1_'></a>[Константность](#toc0_)

## <a id='toc1_1_'></a>[Определение констант](#toc0_)

∙Ключевое слово const позволяет определять типизированные константы.

    double  const pi = 3.1415926535;
    int  const  day_seconds = 24 * 60 * 60;
    
    // массив констант
    int  const  days [12] = {31, 28, 31,
                             30, 31, 30,
                             31, 31, 30,
                             31, 30, 31};

∙Попытка изменить константные данные приводит к неопределённому поведению.
    
    // компилятор не помешает сделать так
    int * may = (int *) &days [4];      // адрес day[4] привести к типу указателя на int
    *may = 30;                          // записать туда значение

\*) во время выполнения кода имеет место UB, т.к. компилятор, зная, что это константа, мог поместить его в область памяти, с защитой от записи или еще как-то оптимизировать код, поэтому результат работы такого кода неопределен (может сработать, а может RuntimeError).

## <a id='toc1_2_'></a>[Указатели и const](#toc0_)

В C++ можно определить как константный указатель, так и указатель на константу:

### <a id='toc1_2_1_'></a>[Указатель на константу](#toc0_)
\*) "указатель на константу" означает, что посредством этого указателя нельзя изменить память, на которую он ссылается. Сам указатель пенеставить на другое место при этом можно, но менять по нему значение все равно нельзя.

    int a = 10;           // НЕ КОНСТАНТА
    const  int * p1 = &a; // указатель на константу
    int  const * p2 = &a; // указатель на константу
    *p1 = 20;             // ошибка (значения переменной нельзя изменить, адрес указателя - можно)
    p2   = 0;             // ОК


### <a id='toc1_2_2_'></a>[Константный указатель](#toc0_)

    int * const p3 = &a;    // константный указатель (значения переменной меняются, адрес указателя - нет)
    *p3 = 30;               // OK
    p3   = 0;               // ошибка

\*) константный указатель ОЧЕНЬ похож на ссылку. Хоть и сказано, что нет арифметики ссылок, но такой пример работает:

        char str[] = { 'M','u','t','a','b','l','e'};
        char & l = str[0];              // потому что ссылка на указатель на char (??)   
        cout << *(&l + 3) << endl;      // OK

        char * const pchar = str;       // указатель на char
        cout << *(pchar + 3) << endl;   // OK    
    
### <a id='toc1_2_3_'></a>[Константный указатель на константу](#toc0_)

    int  const * const p4 = &a;
    *p4 = 30;           // ошибка
    p4   = 0;           // ошибка

\*) арифметика указателей сохраняется:
    
        const char * cstr = "C-style string";
        char const * const const_pchar = cstr;
        cout << *(const_pchar + 3) << endl;

Можно использовать следующее правило: 
### <a id='toc1_2_4_'></a>[“слово const делает неизменяемым тип слева от него”](#toc0_)

    int a = 10;
    int * p = &a;
    
    // указатель на указатель на const int  (ОШИБКА, такое приведение типов запрещено стандартом)
    int  const  **   p1 = &p;   // т.е. неизменяемый a (менять p1 и p можно) - было бы еслиб было можно ))
    
    // указатель на константный указатель на int
    int * const * p2 = &p;      // т.е. неизменяемый p (менять p2 и a можно)
    
    // константный указатель на указатель на int
    int ** const   p3 = &p;     // т.е. неизменяемый p3 (менять p и a можно)

Ошибка заключается следующем присваивании: int const \*\* p1 = &p;  
В этой строчке я ожидаю, что значение типа int\*\* (адрес переменной p) приведётся к значению типа int const\*\*.
Если Вы попробуете скомпилировать этот код, то увидите, что он приводит к ошибке компиляции. Почему? Потому, что стандарт запрещает такие преобразования. Говоря точнее, стандартом разрешены (неявные) преобразования от T\* к T const\*, но не T\*\* к T const\*\*.

Почему бы не разрешить такие преобразования?

Оказывается, что, если бы такие преобразования были разрешены, то можно было бы изменить константные данные. Другими словами, система константности C++ была бы некорректна — она позволяла бы изменять неизменяемые данные.
Вот как это можно сделать:

    int const i = 1;
    int * p = 0;
    // p = &i;  не скомпилируется,  т.к. это преобразование int const * -> int *

    // В этой строке происходит запрещённое преобразование int ** -> int const **
    int const ** pp = &p;  // теперь *pp указывает на переменную p

    // следующая строка скомпилируется, т.к. *pp имеет тип int const*
    *pp = &i; // это соответствует p = &i
    *p = 2; // изменяем значение переменной i 

\*) T\*\*\* => T const\*\*\*  и т.д. также запрещены


## <a id='toc1_3_'></a>[Ссылки и const](#toc0_)

∙Ссылка сама по себе является неизменяемой.

    int a = 10;
    int & const b = a;      // ошибка
    int  const & c = a;     // ссылка на константу
    
∙Использование константных ссылок (на самом деле, ссылок на неизменяемый объект) позволяет избежать копирования объектов (на самом деле, в точности также, как и ссылка без const) при передаче в функцию.

    Point  midpoint(Segment  const & s);    // просто передавая арг.по ссылке - он не защищен от изменений, а так защищен, смысл в этом 
    
∙По константной ссылке можно передавать rvalue. Обычные ссылки инициализируются только lvalue.

    Point p = midpoint(Segment(Point (0,0),     // создать временное значение, проинициализировать (0,0)(1,1)
                               Point (1,1)));   // и передать в функцию. Иначе пришлось бы присваивать промежуточной переменной (*).

\*) т.е. использование константной ссылки продлевает время жизни временной переменной до следующей области видимости путем предохранения ее от удаления со стека, до удаления константной ссылки на нее после реализации принимающей функции

## <a id='toc1_4_'></a>[Константные методы](#toc0_)

∙Методы классов могут быть объявлены как const.

    struct  IntArray {
        size_t  size()  const;      // this будет типа IntArray const *
    };

∙Такие методы не могут менять поля объекта (т.к. тип this — указатель на const).  
∙У константных объектов (через указатель или ссылку на константу) можно вызывать только константные методы:
    
    IntArray  const * p = foo ();
    p->resize (); // ошибка

∙Внутри константных методов можно вызывать только константные методы.

## <a id='toc1_5_'></a>[Две версии одного метода](#toc0_)

∙Слово const является частью сигнатуры метода (нужно указывать и в объявлении и в определении).

    size_t  IntArray ::size() const {return  size_ ;}

∙Можно определить две версии одного метода:

    struct  IntArray {
        int  get(size_t i) const {    // будет работать для константных экземпляров
            return  data_[i];         // возвращает значение
        }
        int & get(size_t i) {         // для остальных
            return  data_[i];         // возвращает ссылку на значение, можно менять
        }
    private:
        size_t  size_;
        int *   data_;
    };

## <a id='toc1_6_'></a>[Cинтаксическая и логическая константность](#toc0_)

∙Синтаксическая константность: константные методы не могут менять поля (обеспечивается компилятором).  
∙Логическая константность — нельзя менять те данные, которые определяют состояние объекта.

    struct  IntArray {
        void  foo() const {                         // синтаксическая константность запрещает изменять сами поля
            // нарушение логической константности (ответвенность разработчика)
            data_ [10] = 1;                         // но т.к. поле data_ это указатель, то поменять по указателю можно, хотя так и не задумывалось
        }
    private:
        size_t  size_;
        int *   data_;
    };

## <a id='toc1_7_'></a>[Ключевое слово mutable](#toc0_)

Ключевое слово mutable позволяет определять поля, которые можно изменять внутри константных методов:

    struct  IntArray {
        size_t  size()  const {
            ++ counter_;                // счетчик обращений ...
            return  size_;
        }
    private:
        size_t  size_;
        int *   data_;
        
        mutable  size_t  counter_;      // ... не должен быть константой
    };



Справедливо также и для объектов, объявленных с квалификатором const:

    struct A
    {
        int field1;
        mutable int field2;
    };
    ...
    const A obj;
    obj.field1 = 0; // Ошибка
    obj.field2 = 0; // ОК



# <a id='toc2_'></a>[Конструктор копирования и оператор присваивания](#toc0_)

## <a id='toc2_1_'></a>[Копирование объектов](#toc0_)

    struct  IntArray {
        ...
    private:
        size_t  size_;
        int *   data_;
    };
    
    int  main() {
        IntArray  a1 (10);  // на стеке будет a1.size_ = 10, a1.data-> массив длины 10 в куче
        IntArray  a2 (20);  // аналогично: 20 и ссылка на массив длины 20
        IntArray  a3 = a1;  // копирование (на стеке a3.size_ == 10, a3.data_ == a1.data_)
        a2 = a1;            // присваивание (a2.size_ == 10, a3.data_ == a1.data_)
        return  0;          // итого не стеке 3 объекта, ссылающиеся на один массив в куче
    }               // при выходе из функции вызовуются деструкторы a3, a2, a1

\*) когда мы создаем объект и сразу же присваиваем это копирование, когда мы создаем объект и не инициализируем его, а инициализируем позже допустим на следущий строчке, тогда происходит присвоение. Суть одна, реализации в объекте разные.

В результате: 

1. деструктор а3 уничтожит a3.data_ -> деструктор a2 попытается освободить свободную память -> RuntimeError
2. из-за того, что при копировании и присваивании забыли освободить память под массивом длины 20 -> утечка памяти

Поэтому нужны специальные механизмы копирования/присваивания

### <a id='toc2_1_1_'></a>[Инструменты - статические анализаторы кода](#toc0_)

Отлавливать подобное могут:
    
    Purify
    Bounds Checker
    Coverity (basically its a code analyzer but, it will catch memory leak in static )
    Glow Code
    dmalloc
    ccmalloc
    NJAMD
    YAMD
    Valgrind
    mpatrol
    Insure++




## <a id='toc2_2_'></a>[Конструктор копирования](#toc0_)

Если не определить конструктор копирования, то он сгенерируется компилятором.

    struct  IntArray {
        // если такая сигнатура (конст.ссылка на объект того же типа) то это конструктор копирования
        IntArray(IntArray  const& a)                        
            : size_(a.size_), data_(new  int[size_ ])   // новый массив
        {
            for (size_t i = 0; i != size_; ++i)
                data_[i] = a.data_[i];                  // копируем
        }
        ...
    private:
        size_t  size_;
        int *   data_;
    };

Вызывается, когда копируется объект, в т.ч. КОГДА ОБЪЕКТ ПЕРЕДАЕТСЯ В ФУНКЦИЮ ПО ЗНАЧЕНИЮ.
Поэтому еще, конструктор копирования принимает именно ссылку на объект, иначе он сам бы вызывал конструктор копирования и зацикливался.

## <a id='toc2_3_'></a>[Оператор присваивания](#toc0_)

Если не определить оператор присваивания, то он тоже сгенерируется компилятором.

    struct  IntArray {
        // если такая сигнатура, и возвращается ссылка на себя же, то это оператор присваивания
        IntArray & operator =( IntArray  const& a)
        {
            if (this != &a) {                           // чтобы не самоликвидироваться при вызове a = a, нужна проверка
                delete  [] data_;                       // т.к. на момент присваивания объект уже существует, то его нужно удалить
                size_ = a.size_;
                data_ = new int[size_];                 // новый
                for (size_t i = 0; i != size_; ++i)
                    data_[i] = a.data_[i];              // копирование
            }
            return *this;                               // ссылка на текущий экземпляр объекта
        }
        ...
    };

\*) a = a врядли может встретиться в коде, но могут быть косвенные присвоения по указателям, которые приведут к такому присвоению и оно бы уничтожило объект в памяти, без такой проверки

## <a id='toc2_4_'></a>[Метод swap](#toc0_)

    struct  IntArray {
        void  swap(IntArray & a) {
            size_t  const t1 = size_;       // меняем местами поле size_
            size_ = a.size_;
            a.size_ = t1;
            
            int * const t2 = data_;         // меняем местами поле data_
            data_ = a.data_;
            a.data_ = t2;
        }
        ...
    private:
        size_t  size_;
        int *   data_;
    };

Можно использовать функцию std::swap из файла algorithm.

    #include  <algorithm >
    
    struct  IntArray {
        void  swap(IntArray & a) {
            std::swap(size_ , a.size_);     // меняем местами поле size_
            std::swap(data_ , a.data_);     // меняем местами поле data_
        }
        ...
    private:
        size_t  size_;
        int *   data_;
    };

### <a id='toc2_4_1_'></a>[* Идиома копирования и обмена](#toc0_)

Всегда стоит избегать дублирования одного и того же кода, так как при изменении или исправлении одного участка, придётся не забыть исправить остальные. Идиома позволяет этого избежать, используя повторно код конструктора копирования.

## <a id='toc2_5_'></a>[Реализация оператора = при помощи swap](#toc0_)

Код выглядит лаконичнее:

    struct  IntArray {
        IntArray(IntArray  const& a)
            : size_(a.size_), data_(new  int[size_ ])       // пишем так, помня про порядок инициализации и порядок полей
        {
            for (size_t i = 0; i != size_; ++i)
                data_[i] = a.data_[i];
        }
        IntArray & operator =( IntArray  const& a) {
            if (this != &a)                 // проверка чисто для эффективности, ошибок не будет, просто чтоб не делать ненужной работы
                IntArray(a).swap(*this);    // как бы tmp = IntArray(a) - создается временный объект, 
                                            // tmp.swap(*this) менятеся с ним полями, tmp самоликвидируется со старыми data_ после исполнения
            return *this;
        }
        ...
    private:
        size_t  size_;
        int *   data_;
    };




## <a id='toc2_6_'></a>[Запрет копирования объектов](#toc0_)

Для некоторых классов понятие "копирование" будет слишком сложным для реализации в одной функции (например, база данных).  
Для того, чтобы запретить копирование, нужно объявить конструктор копирования и оператор присваивания как private и не определять их.

    struct  IntArray {
        ...
    private:
        IntArray(IntArray  const& a);
        IntArray & operator =( IntArray  const& a);
        
        size_t  size_;
        int *   data_;
    };

Если внешний код попробует скопировать объект этого типа, то произойдет ошибка компиляции (т.к. private).  
Если этот код попробует скопировать объект, то произойдет ошибка линковки (т.к. нет определения).


Запись вида: IntArray(IntArray const& a) = delete; отработает аналогичным образом.  
Такой синтаксис появился в стандарте 11-го года. Он работает лучше в том смысле, что обращение к конструктору копирования всегда будет вызывать ошибку компиляции.

## <a id='toc2_7_'></a>[Методы, генерируемые компилятором](#toc0_)

ИТОГО, компилятор генерирует четыре метода:

1. конструктор по умолчанию,  
2. конструктор копирования,  
3. оператор присваивания,  
4. деструктор.

Если потребовалось переопределить конструктор копирования,оператор присваивания или деструктор, то по смыслу нужно переопределить и остальные методы из этого списка. Известно как "правило трёх".

\*) Компилятору C++ позволено выполнять оптимизации, связанные с удалением ненужных вызовов конструктора копирования (copy elision) при определенных условиях, даже если конструктор копирования содержит сайд эффекты (например, вывод в std::cout), поэтому на большинстве современных компиляторов конструктор копирования не будет вызван в большинстве случаем. Однако, компилятор не обязан выполнять эту оптимизацию. Конструктор копирования по прежнему МОЖЕТ быть вызван при передаче объекта в функцию по значению, возврате из функции если внутри он был в роли константы (т.е. когда оригинал уничтожается при завершении функции)

# <a id='toc3_'></a>[Правило трёх (также известное как «Закон Большой Тройки» или «Большая Тройка»)](#toc0_)

— правило в C++, гласящее, что если класс или структура определяет один из следующих методов, то они должны явным образом определить все три метода:

    Деструктор
    Конструктор копирования
    Оператор присваивания копированием

## <a id='toc3_1_'></a>[Правило пяти](#toc0_)

С выходом одиннадцатого стандарта правило расширилось и теперь называется правило пяти. Теперь при реализации конструктора необходимо реализовать:

    Деструктор
    Конструктор копирования
    Оператор присваивания копированием
    Конструктор перемещения                 (копирование с уничтожением оригинала)
    Оператор присваивания перемещением      (присваивание с уничтожением оригинала)

## <a id='toc3_2_'></a>[Правило ноля](#toc0_)

Некоторые сипиписты предлагают радикально отказаться от кастомных конструкторов.  

По этому правилу не стоит определять ни одну из пяти функций самому; надо поручить их создание компилятору (присвоить им значение = default;). Для владения ресурсами вместо простых указателей стоит использовать специальные классы-обёртки, такие как std::unique_ptr и std::shared_ptr.

## <a id='toc3_3_'></a>[Пример реализации правила пяти](#toc0_)

    #include <cstring>

    class RFive
    {
    private:
        char* cstring;

    public:
        // Конструктор со списком инициализации и телом
        RFive(const char* arg)
        : cstring(new char[std::strlen(arg)+1])
        {
            std::strcpy(cstring, arg);
        }

        // Деструктор
        ~RFive()
        {
            delete[] cstring;
        }

        // Конструктор копирования
        RFive(const RFive& other)
        {
            cstring = new char[std::strlen(other.cstring) + 1];
            std::strcpy(cstring, other.cstring);
        }

        // Конструктор перемещения, noexcept - для оптимизации при использовании стандартных контейнеров
        RFive(RFive&& other) noexcept 
        {
            cstring = other.cstring;
            other.cstring = nullptr;
        }

        // Оператор присваивания копированием (copy assignment)
        RFive& operator=(const RFive& other) 
        {
            if (this == &other)
                return *this;

            char* tmp_cstring = new char[std::strlen(other.cstring) + 1];
            std::strcpy(tmp_cstring, other.cstring);
            delete[] cstring;
            cstring = tmp_cstring;
            return *this;
        }

        // Оператор присваивания перемещением (move assignment)
        RFive& operator=(RFive&& other) noexcept
        {
            if (this == &other)
                return *this;

            delete[] cstring;
            cstring = other.cstring;
            other.cstring = nullptr;
            return *this;
        }

    //  Также можно заменить оба оператора присваивания следующим оператором
    //  RFive& operator=(RFive other)
    //  {
    //      std::swap(cstring, other.cstring);
    //      return *this;
    //  }
    };

## <a id='toc3_4_'></a>[Неправильные конструкторы присваивания](#toc0_)

1. нет проверки (this != &other), нет освобождения памяти

        String &operator=(const String &other) 
        {
            str = new char[other.size + 1];
            strcpy(str, other.str);
            size = other.size;
            return *this;
        }

2. нет проверки (this != &other)

        String &operator=(const String &other) 
        {
            delete[] str;
            str = new char[other.size + 1];
            strcpy(str, other.str);
            size = other.size;
            return *this;
        }

3. нет освобождения памяти

        String &operator=(const String &other) 
        {
            if (this != &other) {
                str = other.str;
                size = other.size;
            }
            return *this;
        }

# <a id='toc4_'></a>[Класс массива](#toc0_)

    struct  IntArray {

## <a id='toc4_1_'></a>[/* Поля и конструкторы */](#toc0_)
        explicit IntArray(size_t  size) // запрет неявного преобразования типа IntArray X = 23; 23 попало бы в size_
        : size_(size), data_(new int[size]) 
        {
            for (size_t i = 0; i != size_; ++i)
                data_[i] = 0;
        }
        
        IntArray(IntArray  const& a)   // отличие от конструктора по умолчанию - выделяет память и копирует туда инициализирующий массив
        : size_(a.size_), data_(new  int[size_]) 
        {
            for (size_t i = 0; i != size_; ++i)
                data_[i] = a.data_[i];
        }

## <a id='toc4_2_'></a>[/* Деструктор, оператор присваивания и swap */](#toc0_)
        ~IntArray () 
        {
            delete  [] data_;
        }

        IntArray & operator =( IntArray  const& a)         
        {
            if (this != &a)
                IntArray(a).swap(*this);
            return *this;
        }

        void swap(IntArray & a) 
        {
            std::swap(size_ , a.size_);
            std::swap(data_ , a.data_);
        }
## <a id='toc4_3_'></a>[/* Остальные методы */](#toc0_)
        size_t  size()  const { 
            return  size_; 
        }

        int get(size_t i)   const {             // константное значение (разница только на этапе компиляции, 
            return  data_[i];                   // в рантайме ничего не выбирается), какой get выбрать определяется типом данных
        }
        
        int & get(size_t i)                     // изменяемая ссылка
        {
            return  data_[i];
        }

        void resize(size_t  nsize) 
        {
            IntArray t(nsize );                 // 
            size_t n = nsize > size_ ? size_ : nsize;
            for (size_t i = 0; i != n; ++i)
                t.data_[i] = data_[i];
            swap(t);
        }

    private:
        size_t size_;
        int * data_;
    };